In [ ]:
import torch

# Create a tensor with shape (1, 3, 512, 512)
tensor = torch.randn(1, 3, 512, 512)

# Resize the tensor to (1, 3, 256, 256)
resized_tensor = torch.nn.functional.interpolate(tensor, size=(256, 256), mode='bilinear', align_corners=False)


In [26]:
import torch.nn.functional as F

# assume img is a tensor of shape (5, 512, 512)
img=torch.rand(5,512,512)
print(img.shape)
img_resized = F.interpolate(img.unsqueeze(0), size=(256,256), mode='bilinear', align_corners=False).squeeze()
print(img_resized.shape)


torch.Size([5, 512, 512])
torch.Size([5, 256, 256])


In [34]:
import rasterio
import numpy as np
from PIL import Image
import torchvision.transforms as tt 
from skimage import img_as_float
import torch
#tt.Compose([tt.Resize(224), tt.ToTensor()])    

raster_file='./dataset/train/img/IMG_000001.tif'


def read_img(raster_file: str) -> np.ndarray:
    with rasterio.open(raster_file) as src_img:
        array = src_img.read()
        return array
    
image=read_img(raster_file)
print(f'{type(image)} 1containing {image.dtype} and shape is {image.shape}')
image=img_as_float(image)
print(f'{type(image)} 2containing {image.dtype} and shape is {image.shape}')
image=torch.as_tensor(image, dtype=torch.float32)
print(f'{type(image)} 3containing {image.dtype} and shape is {image.shape}')
image=image.unsqueeze(0)
print(f'{type(image)} 4containing {image.dtype} and shape is {image.shape}')

image = torch.nn.functional.interpolate(image, size=(256, 256), mode='bilinear', align_corners=False)
print(f'{type(image)} 5containing {image.dtype} and shape is {image.shape}')
image=image.squeeze(0)
print(f'{type(image)} 6containing {image.dtype} and shape is {image.shape}')


<class 'numpy.ndarray'> 1containing uint8 and shape is (5, 512, 512)
<class 'numpy.ndarray'> 2containing float64 and shape is (5, 512, 512)
<class 'torch.Tensor'> 3containing torch.float32 and shape is torch.Size([5, 512, 512])
<class 'torch.Tensor'> 4containing torch.float32 and shape is torch.Size([1, 5, 512, 512])
<class 'torch.Tensor'> 5containing torch.float32 and shape is torch.Size([1, 5, 256, 256])
<class 'torch.Tensor'> 6containing torch.float32 and shape is torch.Size([5, 256, 256])


In [8]:
import rasterio
import numpy as np
from PIL import Image
import PIL
import torchvision.transforms as tt 
from skimage import img_as_float
import torch
transforms=tt.Compose([tt.Resize(224), tt.ToTensor()])    

raster_file='./dataset/train/img/IMG_000001.tif'


def read_img(raster_file: str) -> np.ndarray:
    with rasterio.open(raster_file) as src_img:
        array = src_img.read()
        return array
    
image=read_img(raster_file)
print(f'{type(image)} containing {image.dtype} and shape is {image.shape}')
image=img_as_float(image)
print(f'{type(image)} containing {image.dtype} and shape is {image.shape}')
image=transforms(Image.fromarray(np.uint8(image*255)))#.as_tensor(image, dtype=torch.float16)
print(f'{type(image)} containing {image.dtype} and shape is {image.shape}')


<class 'numpy.ndarray'> containing uint8 and shape is (5, 512, 512)
<class 'numpy.ndarray'> containing float64 and shape is (5, 512, 512)


TypeError: Cannot handle this data type: (1, 1, 512), |u1

In [ ]:
import torch
import torchvision.transforms as transforms
import numpy as np

# Create a random 3-channel image of size 100x200
img = np.random.randn(3, 512,512)
#img = torch.rand(3, 100, 200)

img=transforms.ToTensor()(img)
print(img.shape)
# Define the transformation to resize the image to 50x100 (shorter side=50)
transform = transforms.Resize(256)

# Apply the transformation to the image
resized_img = transform(img)
# Check the shape of the resized image tensor
print(resized_img.shape)  # Output: torch.Size([3, 25, 50])


torch.Size([512, 3, 512])


In [3]:
from dataset_module import DatasetModule
from torch.utils.data import DataLoader

ds = DatasetModule("./dataset/train/img/",train='train')
dt = DataLoader(ds, batch_size=1)
for b in dt:
    print(b[0].shape)
    break

type of image <class 'numpy.ndarray'> and dimensions (5, 512, 512)
torch.Size([1, 5, 512, 512])


In [3]:
import rasterio
num_classes=13
raster_file='./raw_dataset/train/D035_2020/Z2_NU/msk/MSK_026068.tif'


def read_msk(raster_file: str) -> np.ndarray:
    with rasterio.open(raster_file) as src_msk:
        array = src_msk.read()[0]
        array[array > num_classes] = num_classes
        array = array-1
        array = np.stack([array == i for i in range(num_classes)], axis=0)
        return array
msk=read_msk(raster_file)

#print(image)

msk=tt.ToTensor()(msk).reshape(512,512,13)
print(f'tensored msk{type(msk)},{msk.shape}')

tensored msk<class 'torch.Tensor'>,torch.Size([512, 512, 13])


In [5]:
# normalize the pixel values -> to be done in dataset_module when reading the image


from skimage import io #numpy array
import imageio
from torchvision.io import read_image
from torchvision.transforms import ToTensor

#new imports
import cv2
import numpy as np 


#### mean subtraction and division by standard deviation using the OpenCV library in Python:

In [8]:
import cv2
import numpy as np 

img_path
img_as_np_array=np.asarray((io.imread(img_path)))
mean, std = cv2.meanStdDev(img_as_np_array)
img_as_np_array = (img_as_np_array - mean) / std
img=ToTensor()(img_as_np_array)


NameError: name 'cv2' is not defined

#### min max using numpy

In [ ]:
import numpy as np

X=np.asarray((io.imread(img_path)))
X = (X - X.min()) / (X.max() - X.min())

In [1]:
from torchlightning_module import torch_lightning_DataModule
from dataset_module import DatasetModule


x=DatasetModule('dataset',train='test')

next(iter(x))

imagepath dataset/train/img/IMG_062613.tif


FileNotFoundError: No such file: '/workspace/semantic-segmentation-competition/dataset/train/img/IMG_062613.tif'

In [ ]:
import os
for i in os.listdir('./dataset/')

In [3]:
import pandas as pd 

df=pd.read_json('./metadata/test_df.jsonl',lines=True)
df[df['image_id']=='IMG_062613.tif'] #'IMG_062613.tif'

,image_id,domain_zone
0,IMG_062613.tif,D012_2019_Z10_UU


In [12]:
def similarity_percentage(list1, list2):
    common_elements = set(list1) & set(list2)
    similarity = len(common_elements) / (len(list1) + len(list2))
    return similarity * 100


In [13]:
x=df['image_id'].to_list()
y=os.listdir('./dataset/test/img')
similarity_percentage(x,y)


15700
15700


In [14]:
print(len(x))
print(len(y))

15700
15700


In [22]:
import pandas as pd 

df_train=pd.read_json('./metadata/train_df.jsonl',lines=True)
df_val=pd.read_json('./metadata/val_df.jsonl',lines=True)
yy=df_train['image_id'].to_list().extend(df_val['image_id'].to_list())
similarity_percentage(os.listdir('./dataset/train/img'),yy)



TypeError: 'NoneType' object is not iterable

In [18]:
similarity_percentage(os.listdir('./dataset/train/img'),df_val['image_id'].to_list())



16.667341840523935

In [13]:

img_path=f'dataset/test/img/{df.iloc[6]["image_id"]}'
img_path

'dataset/test/img/IMG_062619.tif'

In [17]:
from skimage import io

x='/workspace/semantic-segmentation-competition/dataset/train/img/IMG_062613.tif'
io.imread(x)

FileNotFoundError: [Errno 2] No such file or directory: '/workspace/semantic-segmentation-competition/dataset/train/img/IMG_062613.tif'

In [18]:
!ls -l /workspace/semantic-segmentation-competition/dataset/train/img/IMG_062613.tif


ls: cannot access '/workspace/semantic-segmentation-competition/dataset/train/img/IMG_062613.tif': No such file or directory
